In [1]:
import sys

sys.path[1:1] = ["_common", "_common/qiskit"]
sys.path[1:1] = ["../../_common", "../../_common/qiskit"]
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
import time
import math
import numpy as np
np.random.seed(0)
import execute as ex
import metrics as metrics
from collections import defaultdict

In [2]:
from qiskit.quantum_info import SparsePauliOp
from qiskit.primitives import Estimator
from qiskit.quantum_info import Statevector

In [3]:
def HamiltonianSimulation(n_spins, K, t, w, h_x, h_z):
    '''
    Construct a Qiskit circuit for Hamiltonian Simulation
    :param n_spins:The number of spins to simulate
    :param K: The Trotterization order
    :param t: duration of simulation
    :return: return a Qiskit circuit for this Hamiltonian
    '''
    
    # allocate qubits
    qr = QuantumRegister(n_spins); cr = ClassicalRegister(n_spins); qc = QuantumCircuit(qr, cr, name="main")
    tau = t / K

    # start with initial state of 1010101...
    for k in range(0, n_spins, 2):
        qc.x(qr[k])
    qc.barrier()

    # loop over each trotter step, adding gates to the circuit defining the hamiltonian
    for k in range(K):
    
        # the Pauli spin vector product
        [qc.rx(2 * tau * w * h_x[i], qr[i]) for i in range(n_spins)]
        [qc.rz(2 * tau * w * h_z[i], qr[i]) for i in range(n_spins)]
        qc.barrier()
        
        # Use an optimal XXYYZZ combined operator
        # See equation 1 and Figure 6 in https://arxiv.org/pdf/quant-ph/0308006.pdf
        # optimized XX + YY + ZZ operator on each pair of qubits in linear chain
        for j in range(2):
            for i in range(j % 2, n_spins - 1, 2):
                qc.append(xxyyzz_opt_gate(tau).to_instruction(), [qr[i], qr[(i + 1) % n_spins]])

        qc.barrier()

    # measure all the qubits used in the circuit
    for i_qubit in range(n_spins):
        qc.measure(qr[i_qubit], cr[i_qubit])

    return qc

In [17]:
def Hamiltonian_Simulation_TFIM(n_spins, K, t):

    g=0.2 # strength of tranverse field

    # state with initial state of GHZ state: 1/sqrt(2) ( |00...> + |11...> )
    
    qr = QuantumRegister(n_spins); cr = ClassicalRegister(n_spins); qc = QuantumCircuit(qr, cr, name="main")
    tau = t / K

    qc.h(qr[0])
    for k in range(1, n_spins):
        qc.cnot(qr[k-1], qr[k])

    qc.barrier()

    ##calculate TFIM

    for k in range(K):
        # the Pauli spin vector product
        for i in range(n_spins):
            qc.rx(2 * tau * g, qr[i])
        qc.barrier()


        # ZZ operation on each pair of qubits in linear chain
        for j in range(2):
            for i in range(j%2, n_spins-1, 2):
                qc.append(zz_gate(tau).to_instruction(), [qr[i], qr[(i + 1) % n_spins]])
        qc.barrier()


    # reversed tranformation from GHZ state
    for k in reversed(range(1, n_spins)):
        qc.cnot(qr[k-1], qr[k])
    qc.h(qr[0])
    qc.barrier()
        
        # measure all the qubits used in the circuit
    for i_qubit in range(n_spins):
        qc.measure(qr[i_qubit], cr[i_qubit])
        
        
    return qc


In [4]:
# Optimal combined XXYYZZ gate (with double coupling) on q0 and q1 with angle 'tau'
def xxyyzz_opt_gate(tau):
    alpha = tau; beta = tau; gamma = tau
    qr = QuantumRegister(2); qc = QuantumCircuit(qr, name="xxyyzz_opt")
    qc.rz(3.1416/2, qr[1])
    qc.cx(qr[1], qr[0])
    qc.rz(3.1416*gamma - 3.1416/2, qr[0])
    qc.ry(3.1416/2 - 3.1416*alpha, qr[1])
    qc.cx(qr[0], qr[1])
    qc.ry(3.1416*beta - 3.1416/2, qr[1])
    qc.cx(qr[1], qr[0])
    qc.rz(-3.1416/2, qr[0])

    # save circuit example for display
    global XXYYZZ_    
    XXYYZZ_ = qc

    return qc

In [21]:
def zz_gate(tau):
    qr = QuantumRegister(2); qc = QuantumCircuit(qr, name="zz_gate")
    qc.cx(qr[0], qr[1])
    qc.rz(3.1416*tau, qr[1])
    qc.cx(qr[0], qr[1])

    # save circuit example for display
    global ZZ_    
    ZZ_ = qc

    return qc

In [90]:
def Hamiltonian_Simulation_Exact_TFIM(n_spins):
    num_shots = 100000

    qr = QuantumRegister(n_spins); cr = ClassicalRegister(n_spins); qc = QuantumCircuit(qr, cr, name="main")
    g=0.2 # strength of tranverse field

    # state with initial state of GHZ state: 1/sqrt(2) ( |00...> + |11...> )
    qc.h(qr[0])
    for k in range(1, n_spins):
        qc.cnot(qr[k-1], qr[k])
        
    psi = Statevector(qc)
    #print(psi)
        
    #qr2 = QuantumRegister(n_spins); cr2 = ClassicalRegister(n_spins); qc2 = QuantumCircuit(qr2, cr2, name="main")

    ##calculate TFIM

    pauli_list = []
    coeff = (1/((n_spins - 1) + g))
    for i in range(n_spins-1):
        curr_str = "I"*(i)+"ZZ"+"I"*(n_spins-(i+2))
        pauli_list.append((curr_str, coeff))


    
    x_str = "X"*n_spins
    pauli_list.append((x_str, g*coeff))
    pauli_list = SparsePauliOp.from_list(pauli_list)
    
    #print(pauli_list)

    #psi.expectation_value(pauli_list)

    #job = estimator.run([qc], pauli_list, shots = num_shots)
    #x_values = job.result().values
    # print(x_values)
    # print(np.mean(x_values))

    psi2 = psi.evolve(pauli_list)
    
    #print(psi2)

    qr3 = QuantumRegister(n_spins); cr3 = ClassicalRegister(n_spins); qc3 = QuantumCircuit(qr3, cr3, name="main")
    for k in reversed(range(1, n_spins)):
        qc3.cnot(qr3[k-1], qr3[k])
    qc3.h(qr3[0])

    psi3 = psi2.evolve(qc3)
    #print(psi3)

    probs = psi3.probabilities_dict()
    return probs

In [91]:
Hamiltonian_Simulation_Exact_TFIM(3)

{'000': 0.9999999999999996, '001': 5.004680467665246e-34}

In [92]:
import json
from qiskit import execute, Aer
backend = Aer.get_backend("qasm_simulator")

precalculated_data = {}

# parameters
w = 10
k = 3
t = 0.01

# store parameters in precalculated data
precalculated_data['w'] = w
precalculated_data['k'] = k
precalculated_data['t'] = t

# add parameter random values to precalculated data to ensure consistency
np.random.seed(26)
precalculated_data['h_x'] = list(2 * np.random.random(20) - 1) # random numbers between [-1, 1]
np.random.seed(75)
precalculated_data['h_z'] = list(2 * np.random.random(20) - 1) # random numbers between [-1, 1]

num_shots = 100000

for n_spins in range(2,10):
    
    h_x = precalculated_data['h_x'][:n_spins]
    h_z = precalculated_data['h_z'][:n_spins]

    qc = HamiltonianSimulation(n_spins, k, t, w, h_x, h_z)
    
    dist2 = Hamiltonian_Simulation_Exact_TFIM(n_spins)
    
    print(dist2)
    
    qc3 = Hamiltonian_Simulation_TFIM(n_spins, k, t)

    job = execute(qc, backend, shots=num_shots)
    result = job.result()

    counts = result.get_counts(qc)
    
    job3 = execute(qc3, backend, shots = num_shots)
    result3 = job3.result()
    counts3 = result3.get_counts(qc3)
        
    dist = {}
    for key in counts.keys():
        prob = counts[key] / num_shots
        dist[key] = prob
        
#     dist2 = {}
#     for key in counts2.keys():
#         prob = counts2[key] / num_shots
#         dist2[key] = prob
    
    dist3 = {}
    for key in counts3.keys():
        prob = counts3[key] / num_shots
        dist3[key] = prob

    # add dist values to precalculated data for use in fidelity calculation
    precalculated_data[f"Qubits - {n_spins}"] = dist
    
    precalculated_data[f"Qubits2 - {n_spins}"] = dist2
    
    precalculated_data[f"Qubits3 - {n_spins}"] = dist3

# https://stackoverflow.com/questions/36021332/how-to-prettyprint-human-readably-print-a-python-dict-in-json-format-double-q
with open('precalculated_data.json', 'w') as f:
    f.write(json.dumps(
        precalculated_data,
        sort_keys=True,
        indent=4,
        separators=(',', ': ')
        ))

{'00': 1.0, '01': 1.0295220005548027e-34}
{'000': 0.9999999999999996, '001': 5.004680467665246e-34}
{'0000': 0.9999999999999996, '0001': 5.004680467665246e-34}
{'00000': 0.9999999999999996, '00001': 5.004680467665246e-34}
{'000000': 0.9999999999999996, '000001': 5.004680467665246e-34}
{'0000000': 0.9999999999999996, '0000001': 5.004680467665246e-34}
{'00000000': 0.9999999999999993, '00000001': 3.872928019572694e-37}
{'000000000': 1.0, '000000001': 1.0295220005548027e-34}
